In [20]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from imblearn.over_sampling import RandomOverSampler
import numpy as np
from sklearn.model_selection import train_test_split
import os
from sklearn.metrics import precision_score, recall_score, accuracy_score, classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
import joblib

In [5]:
os.getcwd()

'/Users/mahtablashgari/Desktop/GemeinsamRepository/Skin_cancer_diagnosis_tool/Notebooks'

In [7]:
cd ..

/Users/mahtablashgari/Desktop/GemeinsamRepository/Skin_cancer_diagnosis_tool


/Users/mahtablashgari/Library/Python/3.12/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [9]:
cd Dataset/dataverse_files

/Users/mahtablashgari/Desktop/GemeinsamRepository/Skin_cancer_diagnosis_tool/Dataset/dataverse_files


/Users/mahtablashgari/Library/Python/3.12/lib/python/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [21]:
data = pd.read_csv('HAM10000_metadata.csv')
data.head()

,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern


In [22]:
#dx_classes = {'bkl':0, 'df':0, 'nv':0, 'vasc':0, 'akiec':1, 'bcc':1, 'mel':1}
dx_classes = {'nv': 0 , 'mel': 6, 'bkl': 3, 'bcc': 4, 'vasc': 2, 'akiec':5, 'df':1}
data['Class']= data['dx'].map(dx_classes)
data


,lesion_id,image_id,dx,dx_type,age,sex,localization,dataset,Class
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,vidir_modern,3
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,vidir_modern,3
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,vidir_modern,3
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,vidir_modern,3
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,vidir_modern,3
...,...,...,...,...,...,...,...,...,...
10010,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,vidir_modern,5
10011,HAM_0002867,ISIC_0033550,akiec,histo,40.0,male,abdomen,vidir_modern,5
10012,HAM_0002867,ISIC_0033536,akiec,histo,40.0,male,abdomen,vidir_modern,5
10013,HAM_0000239,ISIC_0032854,akiec,histo,80.0,male,face,vidir_modern,5


In [23]:
data['dx'].value_counts()

dx
nv       6705
mel      1113
bkl      1099
bcc       514
akiec     327
vasc      142
df        115
Name: count, dtype: int64

In [24]:
counts = pd.DataFrame(data['dx'].value_counts()).reset_index()
counts

,dx,count
0,nv,6705
1,mel,1113
2,bkl,1099
3,bcc,514
4,akiec,327
5,vasc,142
6,df,115


In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10015 entries, 0 to 10014
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   lesion_id     10015 non-null  object 
 1   image_id      10015 non-null  object 
 2   dx            10015 non-null  object 
 3   dx_type       10015 non-null  object 
 4   age           9958 non-null   float64
 5   sex           10015 non-null  object 
 6   localization  10015 non-null  object 
 7   dataset       10015 non-null  object 
 8   Class         10015 non-null  int64  
dtypes: float64(1), int64(1), object(7)
memory usage: 704.3+ KB


In [26]:
#ohne oversampling:
imputer = SimpleImputer(strategy='mean')
data['age'] = imputer.fit_transform(data[['age']])

label_encoder_sex = LabelEncoder()
data['sex'] = label_encoder_sex.fit_transform(data['sex'])

label_encoder_localization = LabelEncoder()
data['localization'] = label_encoder_localization.fit_transform(data['localization'])

X = data[['age', 'sex', 'localization']]  
y = data['Class']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_RF_ohneOverSampling = RandomForestClassifier(random_state=42)
model_RF_ohneOverSampling.fit(X_train, y_train)

y_pred = model_RF_ohneOverSampling.predict(X_test)
y_pred_proba = model_RF_ohneOverSampling.predict_proba(X_test)

# Auswertung
print("Accuracy:", accuracy_score(y_test, y_pred))
print("classifikation report:")
print(classification_report(y_test, y_pred))

print("Vorhersagewahrscheinlichkeiten für die ersten 5 Beispiele:")
print(y_pred_proba[:5])


Genauigkeit: 0.7059410883674488
Klassifikationsbericht:
              precision    recall  f1-score   support

           0       0.76      0.97      0.85      1338
           1       0.00      0.00      0.00        28
           2       1.00      0.05      0.09        21
           3       0.44      0.33      0.38       228
           4       0.30      0.09      0.13        93
           5       0.25      0.09      0.13        69
           6       0.34      0.10      0.16       226

    accuracy                           0.71      2003
   macro avg       0.44      0.23      0.25      2003
weighted avg       0.63      0.71      0.64      2003

Vorhersagewahrscheinlichkeiten für die ersten 5 Beispiele:
[[0.         0.         0.         0.37865315 0.21771033 0.11675797
  0.28687855]
 [0.87753324 0.01096401 0.         0.01004333 0.01208969 0.
  0.08936974]
 [0.477528   0.         0.         0.11464392 0.07939528 0.01676595
  0.31166684]
 [0.9249331  0.         0.03755863 0.01839865 0.  

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  

In [27]:

from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score


y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6])

roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class="ovr")
print(f"ROC-AUC-Score: {roc_auc}")

ROC-AUC-Score: 0.8288085038901901


In [28]:
joblib.dump(model_RF_ohneOverSampling, 'random_forest_model_ohneOversampling.pkl')
print("Modell erfolgreich gespeichert!")

Modell erfolgreich gespeichert!


In [65]:
#mit oversampling
imputer = SimpleImputer(strategy='mean')
data['age'] = imputer.fit_transform(data[['age']])


label_encoder_sex = LabelEncoder()
data['sex'] = label_encoder_sex.fit_transform(data['sex'])

label_encoder_localization = LabelEncoder()
data['localization'] = label_encoder_localization.fit_transform(data['localization'])

X = data[['age', 'sex', 'localization']] 
y = data['Class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Oversampling 
oversample = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversample.fit_resample(X_train, y_train)

model_RF_OverSampling = RandomForestClassifier(random_state=42)
model_RF_OverSampling.fit(X_train_resampled, y_train_resampled)

y_pred = model_RF_OverSampling.predict(X_test)
y_pred_proba = model_RF_OverSampling.predict_proba(X_test)

print("Genauigkeit:", accuracy_score(y_test, y_pred))
print("Klassifikationsbericht:")
print(classification_report(y_test, y_pred))

y_test_binarized = label_binarize(y_test, classes=[0, 1, 2, 3, 4, 5, 6])

# ROC-AUC-Score für jede Klasse berechnen (One-vs-Rest)
roc_auc = roc_auc_score(y_test_binarized, y_pred_proba, multi_class="ovr")
print(f"ROC-AUC-Score: {roc_auc}")

vergleich = pd.DataFrame({
    'Tatsächlicher Wert': y_test,
    'Vorhergesagter Wert': y_pred
})

for i in range(len(y_test_binarized[0])):
    vergleich[f'Wahrscheinlichkeit Klasse {i}'] = y_pred_proba[:, i]

# Die ersten 10 Vergleiche anzeigen
print(vergleich.head(10))


Genauigkeit: 0.33349975037443835
Klassifikationsbericht:
              precision    recall  f1-score   support

           0       0.92      0.36      0.52      1338
           1       0.04      0.57      0.08        28
           2       0.04      0.62      0.07        21
           3       0.32      0.19      0.24       228
           4       0.14      0.40      0.21        93
           5       0.17      0.51      0.25        69
           6       0.22      0.16      0.19       226

    accuracy                           0.33      2003
   macro avg       0.26      0.40      0.22      2003
weighted avg       0.69      0.33      0.42      2003

ROC-AUC-Score: 0.791708619545466
      Tatsächlicher Wert  Vorhergesagter Wert  Wahrscheinlichkeit Klasse 0  \
1617                   6                    4                     0.000000   
8128                   0                    0                     0.376188   
2168                   6                    6                     0.108812   
1

In [67]:
joblib.dump(model_RF_OverSampling, 'random_forest_model_Oversampling.pkl')
print("Modell erfolgreich gespeichert!")

Modell erfolgreich gespeichert!
